In [3]:
from sherpa.astro import ui
import numpy
import os

ui.load_pha('data/bug-149/acisf00308_000N001_r0044_pha3.fits.gz', use_errors=True)

# test case 1: expected results when grouping the background independently from the signal

data = ui.get_data()

data.group_counts(16)
data.group_counts(num=10, bkg_id=1)

# what should happen? How does that affect the data?
# also notice how this doesn't work... see the docs at http://cxc.harvard.edu/sherpa/ahelp/group_sherpa.html
# or help(group_counts)


read ARF file data/bug-149/acisf00308_000N001_r0044_arf3.fits
read RMF file data/bug-149/acisf00308_000N001_r0044_rmf3.fits
read background file data/bug-149/acisf00308_000N001_r0044_pha3.fits


TypeError: group_counts() got an unexpected keyword argument 'bkg_id'

In [66]:
# test case 2: group_snr_adapt()
ui.ungroup()
ui.group_adapt_snr(3)

print ui.get_data().get_dep(filter=True)

# what do we expect from adaptive grouping?



[  5.   7.   0.   7.   1.   9.   9.   9.   3.  12.  11.   7.   3.  11.   6.
  10.  10.  12.  13.   9.   9.   3.  10.   3.   9.   2.   8.   3.   7.   9.
   8.   9.   9.   1.   9.   7.   9.   7.   7.   5.   8.   8.   8.   7.   0.
   8.   5.   8.   1.   7.   1.   6.   5.   7.   2.   9.   5.   8.   1.   9.
   5.   7.   4.   8.   8.   8.   9.   6.   1.   6.   6.   6.   2.  10.   3.]


In [6]:
# test case 3: Nick Lee's bug
nlee_file = '/stage/pool7/jbudynk/bug_reports/nlee_grouping/18076_bin5.pi' # proprietary - not in repo
ui.clean()
ui.load_pha(nlee_file)

rmf = ui.get_rmf()
arf = ui.get_arf()
bkg_scale = ui.get_bkg_scale()

# the data has already been grouped
data = ui.get_data()

print "\n data before ignore_bad() is called"
print data.get_dep(filter=True)

# ignore bad data
data.ignore_bad()

print "\n data after ignore_bad() is called"
print data.get_dep(filter=True)

# now regroup the data
data.ungroup()
data.group_counts(5)

print "\n ungrouped the data, then regrouped in Sherpa"
print data.get_dep(filter=True)

# ignore any bad data
data.ignore_bad()

print "\n after calling ignore_bad() on dataset"
print data.get_dep(filter=True)

# notice that the arrays are different. What should happen instead?
# We can take a look at Omar's email.

read ARF file /stage/pool7/jbudynk/bug_reports/nlee_grouping/18076.corr.arf
read RMF file /stage/pool7/jbudynk/bug_reports/nlee_grouping/18076.rmf
read ARF (background) file /stage/pool7/jbudynk/bug_reports/nlee_grouping/18076_bkg.arf
read RMF (background) file /stage/pool7/jbudynk/bug_reports/nlee_grouping/18076_bkg.rmf
read background file /stage/pool7/jbudynk/bug_reports/nlee_grouping/18076_bkg.pi

 data before ignore_bad() is called
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  2.  1.
  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  1.  0.  0.  5.
  6.  6.  7.  5.  6.  6.  5.  5.  6.  5.  5.  5.  5.  5.  6.  5.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.

In [5]:
# issue-149. See https://github.com/sherpa/sherpa/issues/149 for more info.
# It's very similar, if not the same issue as Nick's bug report.

ui.load_pha(0,"data/bug-149/acisf00308_000N001_r0044_pha3.fits.gz", use_errors=True)

data = ui.get_data(0)

#filter and group data
data.notice(0.5, 7.0)
data.group_counts(16)

# the expected grouped counts after filtering
new_y = [19, 18, 16, 21, 18, 19, 16, 17, 19, 16, 16, 17, 16, 17,
         16, 17, 16, 16, 16, 17, 16, 16, 16, 16, 16, 16, 16]

print "\n expected grouped and filtered array: "
print new_y

# the array after grouping and filtering
print "\n actual grouped and filtered array: "
print data.get_dep(filter=True)

# what's happening is that group_counts() is ignoring the filter.
# instead, I think it should take the filter into consideration before grouping
# then group only the data within the notice range.

# if we ignore the bad data points with ignore_bad(), we get the warning Nick's
# bug misses to show
print "\n after ignore_bad(): "
data.ignore_bad()
print data.get_dep(filter=True)

read ARF file data/bug-149/acisf00308_000N001_r0044_arf3.fits
read RMF file data/bug-149/acisf00308_000N001_r0044_rmf3.fits
read background file data/bug-149/acisf00308_000N001_r0044_pha3.fits

 expected grouped and filtered array: 
[19, 18, 16, 21, 18, 19, 16, 17, 19, 16, 16, 17, 16, 17, 16, 17, 16, 16, 16, 17, 16, 16, 16, 16, 16, 16, 16]

 actual grouped and filtered array: 
[ 17.  16.  16.  17.  17.  17.  16.  16.  17.  17.  19.  16.  16.  17.  16.
  17.  16.  17.  16.  16.  16.  17.  16.  16.  16.  16.  16.  16.  16.  13.]

 after ignore_bad(): 
[ 17.  16.  16.  17.  17.  17.  16.  16.  17.  17.  19.  16.  16.  17.  16.
  17.  16.  17.  16.  16.  16.  17.  16.  16.  16.  16.  16.  16.  16.]
